# Best bus stops

Find the best bus stops from PTV data.


Many bus routes don't have night services. So I'd like to find the bus stops that have night services.

In [1]:
import components as comp
import pandas as pd
import numpy as np

In [2]:
# Utilize a custom module to convert PTV's gtfs.zip data into a Series of Pandas DataFrames

df = comp.process_gtfs_zip('http://data.ptv.vic.gov.au/downloads/gtfs.zip', '')
# 2m30s

In [3]:
ptv = df.set_index(['branch_id', 'table_name'], inplace=False)['df']

In [4]:
# Create a DataFrame for each bus table in the GTFS data
df_bus_agency : pd.DataFrame = ptv['4']['agency']
df_bus_calendar : pd.DataFrame = ptv['4']['calendar']
df_bus_calendar_dates : pd.DataFrame = ptv['4']['calendar_dates']
df_bus_routes : pd.DataFrame = ptv['4']['routes']
df_bus_shapes : pd.DataFrame = ptv['4']['shapes']
df_bus_stops : pd.DataFrame = ptv['4']['stops']
df_bus_stop_times : pd.DataFrame = ptv['4']['stop_times']
df_bus_trips : pd.DataFrame = ptv['4']['trips']

In [5]:
def get_max_row(df : pd.DataFrame, by: str):
    """
    Return the row with the maximum/minimum value of 'by' column in the given dataframe

    df.max() and df.min() only returns the considered column instead of the whole row.
    """
    return df.loc[df[by].idxmax()]

def get_min_row(df : pd.DataFrame, by: str):
    """
    Return the row with the maximum/minimum value of 'by' column in the given dataframe

    df.max() and df.min() only returns the considered column instead of the whole row.
    """
    return df.loc[df[by].idxmin()]

In [6]:
days_in_week = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']

In [7]:
dfbst = pd.merge(df_bus_stop_times, df_bus_trips, on='trip_id', how='left')
dfbst = pd.merge(dfbst, df_bus_calendar, on='service_id', how='left')
dfbst = dfbst[['stop_id', 'trip_id', 'route_id', 'arrival_time', 'start_date', 'end_date'] + days_in_week]

# 10s

In [8]:
dkx = {}
for d in days_in_week:
    dkx[d] = dfbst[dfbst[d] == 1].groupby(['stop_id', 'route_id'])['arrival_time'].max()
    dkx[d].rename(d, inplace=True)

# 10s


In [13]:
dfbs = pd.merge(dkx['monday'], dkx['tuesday'], on=['stop_id', 'route_id'], how='outer')
for d in days_in_week[2:]:
    dfbs = pd.merge(dfbs, dkx[d], on=['stop_id', 'route_id'], how='outer')
dfbs.dropna(inplace=True)
dfbs['weekday'] = dfbs[['monday', 'tuesday', 'wednesday', 'thursday', 'friday']].min(axis=1)
dfbs['weekend'] = dfbs[['saturday', 'sunday']].min(axis=1)
dfbs['week'] = dfbs.min(axis=1)
dfbs.sort_values(by='week', inplace=True, ascending=False)
dfbs.reset_index(inplace=True)

In [14]:
dfbs

,stop_id,route_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,weekday,weekend,week
0,21048,14-907-aus-1,24:52:00,24:52:00,24:52:00,24:52:00,26:42:00,27:37:00,24:43:00,24:52:00,24:43:00,24:43:00
1,47684,14-907-aus-1,24:51:00,24:51:00,24:51:00,24:51:00,26:41:00,27:36:00,24:42:00,24:51:00,24:42:00,24:42:00
2,44914,14-908-aus-1,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:42:00,24:50:00,24:42:00,24:42:00
3,3965,14-907-aus-1,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:41:00,24:50:00,24:41:00,24:41:00
4,3966,14-907-aus-1,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:41:00,24:50:00,24:41:00,24:41:00
...,...,...,...,...,...,...,...,...,...,...,...,...
28039,5182,17-908-aus-1,22:47:00,22:47:00,22:47:00,22:47:00,22:47:00,05:11:00,06:06:00,22:47:00,05:11:00,05:11:00
28040,5183,17-908-aus-1,22:47:00,22:47:00,22:47:00,22:47:00,22:47:00,05:11:00,06:06:00,22:47:00,05:11:00,05:11:00
28041,4385,17-908-aus-1,22:46:00,22:46:00,22:46:00,22:46:00,22:46:00,05:10:00,06:05:00,22:46:00,05:10:00,05:10:00
28042,4387,17-908-aus-1,22:45:00,22:45:00,22:45:00,22:45:00,22:45:00,05:09:00,06:04:00,22:45:00,05:09:00,05:09:00


In [15]:
dfbsi = pd.merge(dfbs, df_bus_stops, on='stop_id', how='left')
dfbsi = pd.merge(dfbsi, df_bus_routes, on='route_id', how='left')

In [16]:
dfbsi.head(10)

,stop_id,route_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,weekday,...,week,stop_name,stop_lat,stop_lon,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,21048,14-907-aus-1,24:52:00,24:52:00,24:52:00,24:52:00,26:42:00,27:37:00,24:43:00,24:52:00,...,24:43:00,Mitcham Station/Colombo St (Mitcham),-37.818058,145.192188,NaN,907,Mitcham - City (King/Lonsdale Sts),3,FF8200,FFFFFF
1,47684,14-907-aus-1,24:51:00,24:51:00,24:51:00,24:51:00,26:41:00,27:36:00,24:42:00,24:51:00,...,24:42:00,Harrison St/Mitcham Rd (Mitcham),-37.813988,145.193560,NaN,907,Mitcham - City (King/Lonsdale Sts),3,FF8200,FFFFFF
2,44914,14-908-aus-1,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:42:00,24:50:00,...,24:42:00,The Pines SC/Reynolds Rd (Doncaster East),-37.762878,145.168921,NaN,908,The Pines SC - City (King/Lonsdale Sts),3,FF8200,FFFFFF
3,3965,14-907-aus-1,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:41:00,24:50:00,...,24:41:00,Burnett St/Mitcham Rd (Mitcham),-37.812065,145.193368,NaN,907,Mitcham - City (King/Lonsdale Sts),3,FF8200,FFFFFF
4,3966,14-907-aus-1,24:50:00,24:50:00,24:50:00,24:50:00,26:40:00,27:35:00,24:41:00,24:50:00,...,24:41:00,Doncaster East Rd/Mitcham Rd (Mitcham),-37.809411,145.193081,NaN,907,Mitcham - City (King/Lonsdale Sts),3,FF8200,FFFFFF
5,4019,14-908-aus-1,24:48:00,24:48:00,24:48:00,24:48:00,26:38:00,27:33:00,24:40:00,24:48:00,...,24:40:00,Serpells Rd/Blackburn Rd (Doncaster East),-37.769142,145.165486,NaN,908,The Pines SC - City (King/Lonsdale Sts),3,FF8200,FFFFFF
6,4020,14-908-aus-1,24:49:00,24:49:00,24:49:00,24:49:00,26:39:00,27:34:00,24:40:00,24:49:00,...,24:40:00,Deep Creek Anglican Church/Blackburn Rd (Donca...,-37.766806,145.165918,NaN,908,The Pines SC - City (King/Lonsdale Sts),3,FF8200,FFFFFF
7,3967,14-907-aus-1,24:49:00,24:49:00,24:49:00,24:49:00,26:39:00,27:34:00,24:40:00,24:49:00,...,24:40:00,Mullauna Secondary College/241 Mitcham Rd (Mit...,-37.807323,145.192087,NaN,907,Mitcham - City (King/Lonsdale Sts),3,FF8200,FFFFFF
8,4018,14-908-aus-1,24:48:00,24:48:00,24:48:00,24:48:00,26:38:00,27:33:00,24:39:00,24:48:00,...,24:39:00,Zerbe Ave/Blackburn Rd (Doncaster East),-37.772072,145.164959,NaN,908,The Pines SC - City (King/Lonsdale Sts),3,FF8200,FFFFFF
9,3968,14-907-aus-1,24:48:00,24:48:00,24:48:00,24:48:00,26:38:00,27:33:00,24:39:00,24:48:00,...,24:39:00,Chippewa Ave/Mitcham Rd (Mitcham),-37.804984,145.190008,NaN,907,Mitcham - City (King/Lonsdale Sts),3,FF8200,FFFFFF
